In [1]:
import pandas as pd
import numpy as np

solar_pred = pd.read_excel(r"C:\Users\tomas\Downloads\Objective1_Solar_Prediction.xlsx", parse_dates=['Datetime'])
actual_used = pd.read_csv(r"C:\Users\tomas\Downloads\repsol_dataset\Consumo_fotovoltaica.csv", parse_dates=['FECHA'])
actual_used.rename(columns={'FECHA': 'Datetime', 'TOTAL_KWH_ENERGIA': 'ACTUAL_USED'}, inplace=True)

In [2]:
grid_demand = pd.read_csv(r"C:\Users\tomas\Downloads\repsol_dataset\Consumo.csv", parse_dates=['FECHA'])
grid_demand.rename(columns={'FECHA': 'Datetime', 'TOTAL_KWH_ENERGIA': 'GRID_CONSUMED'}, inplace=True)

In [8]:
carbon = pd.read_csv(r"C:\Users\tomas\Downloads\repsol_dataset\ES_2024_hourly.csv", parse_dates=['Datetime (UTC)'])
carbon = carbon[carbon['Country'] == 'Spain']
carbon = carbon[['Datetime (UTC)', 'Carbon Intensity gCO₂eq/kWh (direct)']]
carbon.rename(columns={'Carbon Intensity gCO₂eq/kWh (direct)': 'CARBON_INTENSITY'}, inplace=True)

In [ ]:
solar_pred['Datetime'] = solar_pred['Datetime'].dt.tz_localize(None)
actual_used['Datetime'] = actual_used['Datetime'].dt.tz_localize(None)
grid_demand['Datetime'] = grid_demand['Datetime'].dt.tz_localize(None)
carbon['Datetime'] = carbon['Datetime (UTC)'].dt.tz_localize(None)

df = solar_pred.merge(actual_used, on='Datetime', how='left')
df = df.merge(grid_demand, on='Datetime', how='left')
df = df.merge(carbon, on='Datetime', how='left')

In [15]:
# Calculate EXCESS solar
df['EXCESS_SOLAR'] = df['KWH_ENERGIA'] - df['ACTUAL_USED']
df['EXCESS_SOLAR'] = df['EXCESS_SOLAR'].clip(lower=0)

In [16]:
# Prepare tracking columns
df['ENERGY_CHARGED'] = 0.0
df['ENERGY_DISCHARGED'] = 0.0
df['CO2_AVOIDED'] = 0.0

In [26]:
df = df.fillna(0)

In [27]:
# Simulate day-by-day energy charging and discharging
for day, day_df in df.groupby(df['Datetime'].dt.date):  # Loop through each unique date
    # Sort the day’s data chronologically
    day_df = day_df.sort_values('Datetime')
    
    # Copy available excess solar energy for this day
    charge_available = day_df['EXCESS_SOLAR'].copy()
    
    charged = 0  # Track total energy charged for the day

    # Loop over each hour to simulate charging
    for i in range(len(day_df)):
        if charged >= 100:  # Maximum daily battery capacity is 100 units
            break
        # Determine how much can be charged in this hour
        charge_possible = min(100 - charged, 100, charge_available.iloc[i])
        
        # Store the charged energy in the main DataFrame
        df.loc[day_df.index[i], 'ENERGY_CHARGED'] = charge_possible
        
        # Accumulate the total charged energy
        charged += charge_possible

    # Identify hours with the highest carbon intensity first for discharging
    discharge_hours = day_df.sort_values('CARBON_INTENSITY', ascending=False)
    
    discharged = 0  # Track total energy discharged for the day

    # Loop through hours in descending carbon intensity to maximize CO2 avoided
    for i in discharge_hours.index:
        if discharged >= 100:  # Maximum daily discharging limit is 100 units
            break
        # Determine how much of the grid consumption can be replaced (discharged)
        grid_need = df.loc[i, 'GRID_CONSUMED']
        
        # Discharge either what's needed or what's remaining in capacity
        discharge = min(100 - discharged, grid_need)
        
        # Store discharged energy and corresponding CO2 avoided
        df.loc[i, 'ENERGY_DISCHARGED'] = discharge
        df.loc[i, 'CO2_AVOIDED'] = discharge * df.loc[i, 'CARBON_INTENSITY']
        
        # Accumulate total discharged energy
        discharged += discharge


In [28]:
# Final metrics
total_generated = df['KWH_ENERGIA'].sum()
total_direct_used = df['ACTUAL_USED'].sum()
total_battery_used = df['ENERGY_DISCHARGED'].sum()

# 🔢 Ra (Self-Consumption Ratio)
Ra = round(((total_direct_used + total_battery_used) / total_generated) * 100, 4)
print(" Ra (Self-Consumption Ratio %):", Ra)

# Save CO2_Avoided output file (for Objective 3 too)
df_out = df[['Datetime', 'CO2_AVOIDED']]
df_out

 Ra (Self-Consumption Ratio %): 117.4937


,Datetime,CO2_AVOIDED
0,2024-09-01 00:00:00,0.0
1,2024-09-01 01:00:00,0.0
2,2024-09-01 02:00:00,0.0
3,2024-09-01 03:00:00,0.0
4,2024-09-01 04:00:00,0.0
...,...,...
715,2024-09-30 19:00:00,0.0
716,2024-09-30 20:00:00,0.0
717,2024-09-30 21:00:00,0.0
718,2024-09-30 22:00:00,0.0
